In [1]:
%reset -s -f
%load_ext autoreload
%autoreload 2

In [5]:
import sys 
sys.path.append("./../")
import pandas as  pd

In [39]:


toydata = dict(
    cola=[1,1,1,1,3,2,3,5],
    colb=[3,1,3,1,2,2,4,4],
    colc=[3,1,3,3,2,2,4,4],
    target=["a","b","a","b","c","a","c","b"]
)
df1 = pd.DataFrame(toydata)


In [7]:
from src.node import get_weighted_gini_impurity , make_gini_attributes , get_min_giniattributes


In [42]:
attribute_names =  ['age', 'income','student', 'credit_rate']
class_name = 'default'
data1 ={
    'age' : ['youth', 'youth', 'middle_age', 'senior', 'senior', 'senior','middle_age', 'youth', 'youth', 'senior', 'youth', 'middle_age','middle_age', 'senior'],
    'income' : ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium','low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student' : ['no','no','no','no','yes','yes','yes','no','yes','yes','yes','no','yes','no'],
    'credit_rate' : ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair','excellent', 'excellent', 'fair', 'excellent'],
    'default' : ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes','yes', 'yes', 'yes', 'no']
}
df1 = pd.DataFrame (data1, columns=data1.keys())


target_col = "income"
cols = ['age', 'default','student', 'credit_rate']
result = make_gini_attributes(df1 , cols =cols,target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)

{'age': 0.5785714285714285, 'default': 0.6412698412698412, 'student': 0.5306122448979592, 'credit_rate': 0.6369047619047619}
student


In [43]:
select_col

'student'

In [44]:
df1.groupby([select_col,target_col]).size()

student  income
no       high      3
         medium    4
yes      high      1
         low       4
         medium    2
dtype: int64

In [45]:
#df1[]
print(df1[select_col].unique())


['no' 'yes']


In [48]:
#cols = ['age', 'default', 'credit_rate']

selected_df1 = df1.query("student == 'no'")

result = make_gini_attributes(selected_df1 , cols = ['age', 'default', 'credit_rate'],target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)
if get_min_giniattributes(result)[1] == 0.0 :
    print("finish split")

{'age': 0.3333333333333333, 'default': 0.47619047619047616, 'credit_rate': 0.47619047619047616}
age


In [52]:
selected_df1 = df1.query("student == 'no'")

list(selected_df1["age"].unique())


selected_df2 = df1.query("student == 'no' & age == 'youth'")
result = make_gini_attributes(selected_df2 , cols = ['default', 'credit_rate'],target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)
if get_min_giniattributes(result)[1] == 0.0 :
    print("finish split")



{'default': 0.4444444444444444, 'credit_rate': 0.3333333333333333}
credit_rate


In [36]:
import numpy as np
selected_df1['colb'].unique().tolist()
selected_df1.groupby(['target']).apply(lambda x : {"colb" : list(np.unique(x['colb']))})

target
a    {'colb': [3]}
b    {'colb': [1]}
dtype: object

In [38]:
selected_df1

,cola,colb,colc,target
5,2,2,2,a


In [140]:
from sklearn.model_selection import ParameterGrid

ASET = dict(
    cola = ["A"],
    colb = [1,2,3],
    colc = ["a","c"]
)
BSET = dict(
    cola = ["A"],
    colb = [2,3,4,5],
    colc = ["b","d"]
)
CSET = dict(
    cola = ["C"],
    colb = [1,4],
    colc = ["a","b"]
)
DSET = dict(
    cola = ["B"],
    colb = [1,2],
    colc = ["e","b"]
)
ESET = dict(
    cola = ["B"],
    colb = [1,2],
    colc = ["a","d"]
)


RESULT = []
RESULT.extend(list(ParameterGrid(ASET)))
RESULT.extend(list(ParameterGrid(BSET)))
RESULT.extend(list(ParameterGrid(CSET)))
RESULT.extend(list(ParameterGrid(DSET)))
RESULT.extend(list(ParameterGrid(ESET)))
def find_unique_dict(RESULT) :
    RESULT = list(map(dict, set(tuple(sorted(d.items())) for d in RESULT)))
    return RESULT
from collections import Counter
assert len(find_unique_dict(RESULT)) == len(RESULT)
comb = pd.DataFrame()
df = pd.DataFrame(list(ParameterGrid(ASET)))
df["target"] = "aa"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(BSET)))
df["target"] = "bb"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(CSET)))
df["target"] = "cc"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(DSET)))
df["target"] = "dd"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(ESET)))
df["target"] = "ee"
comb = pd.concat([comb,df],axis=0)
comb = comb.reset_index(drop=True)
comb

In [185]:
def fit(comb , cols , target_col, previous_condition=[]) :
    result = make_gini_attributes(comb , cols =cols,target_col=target_col)
    select_col = get_min_giniattributes(result)[0]
    minthreshold = get_min_giniattributes(result)[1]
    
    find_pure_result = dict(filter(lambda x : x[1] == 0.0 , result.items()))
    if len(find_pure_result) == 0 :
        select_candidate = list(comb[select_col].unique())
        qs = [f"{select_col} == '{i}'" for i in select_candidate]        
        qs_dict = [{select_col : [i]} for i in select_candidate]        
        
        _cols = cols 
        for idx , q in enumerate(qs) :
            _comb = comb.query(q)
            _cols = list(set(_cols).difference(set([select_col])))
            previous_condition.append(qs_dict[idx])
            fit(_comb , _cols , target_col,previous_condition)
    else :
        for select_col, gini_impurity in find_pure_result.items() : 
            r = comb.groupby([target_col]).apply(lambda x : {select_col:list(x[select_col].unique())})
            r= r.to_dict()
            for k , v in r.items() :
                for cond in previous_condition:
                    v.update(cond)
            print("---")
            print(r)
            print("---")
        else :
            del previous_condition[-1]

fit(comb , cols ,"target")

target_aa = dict(cola=["A"],colc=["a","c"])
target_bb = dict(cola=["A"],colc=["b","d"])
target_cc = dict(cola=["C"],colc=["a","b"])
target_cc = dict(cola=["C"],colb=[1,4])
target_dd = dict(cola=["B"],colb=[1,2])
target_dd = dict(cola=["B"],colc=["e","b"])
target_ee = dict(cola=["B"],colc=["a","d"])

---
{'aa': {'colc': ['a', 'c'], 'cola': ['A']}, 'bb': {'colc': ['b', 'd'], 'cola': ['A']}}
---
---
{'cc': {'colc': ['a', 'b'], 'cola': ['C']}}
---
---
{'cc': {'colb': [1, 4], 'cola': ['C']}}
---
---
{'dd': {'colc': ['e', 'b'], 'cola': ['B']}, 'ee': {'colc': ['a', 'd'], 'cola': ['B']}}
---


In [473]:
from re import I
from sklearn.model_selection import ParameterGrid

ASET = dict(
    cola = ["A"],
    colb = [1,2,3],
    colc = ["a","c"],
    cold = ["#","###"]
)
BSET = dict(
    cola = ["A"],
    colb = [2,3,4,5],
    colc = ["b","d"],
        cold = ["#","###"]

)
CSET = dict(
    cola = ["C"],
    colb = [1,4],
    colc = ["a","b"],
        cold = ["#","###"]

)
DSET = dict(
    cola = ["B"],
    colb = [1,2],
    colc = ["e","b"],
    cold = ["#","###"]

)
ESET = dict(
    cola = ["B"],
    colb = [3,4],
    colc = ["a","b"],
    cold = ["#","###"]

)


RESULT = []
RESULT.extend(list(ParameterGrid(ASET)))
RESULT.extend(list(ParameterGrid(BSET)))
RESULT.extend(list(ParameterGrid(CSET)))
RESULT.extend(list(ParameterGrid(DSET)))
RESULT.extend(list(ParameterGrid(ESET)))
def find_unique_dict(RESULT) :
    RESULT = list(map(dict, set(tuple(sorted(d.items())) for d in RESULT)))
    return RESULT
from collections import Counter
# assert len(find_unique_dict(RESULT)) == len(RESULT)
comb = pd.DataFrame()
df = pd.DataFrame(list(ParameterGrid(ASET)))
df["target"] = "aa"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(BSET)))
df["target"] = "bb"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(CSET)))
df["target"] = "cc"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(DSET)))
df["target"] = "dd"
comb = pd.concat([comb,df],axis=0)
df = pd.DataFrame(list(ParameterGrid(ESET)))
df["target"] = "ee"
comb = pd.concat([comb,df],axis=0)
comb = comb.reset_index(drop=True)
comb

class SeperateCode(object) :
    def __init__(self,) :
        pass

    def make_where_query(self,condition_list:dict) :
        total_q = []
        for k , v in condition_list.items() :
            if len(v) == 1 :
                q = f"{k} == '{v[0]}'" 
            else :
                _comb = list(v)
                q = f"{k} in {_comb}"
            total_q.append(q)
        else :
            total_q = " and ".join([f"({i})" for i in total_q])
        return total_q

    def print_pretty(self , d ,indent=0) :
        for key, value in d.items():
            print('\t' * indent + str(key))
            if isinstance(value, dict):
                self.print_pretty(value, indent+1)
            else:
                print('\t' * (indent+1) + str(value))

    def initialize(self,comb,target_col) :
        self._comb = comb
        self._target_index_dict_list = comb.groupby([target_col]).apply(lambda x : x.index.tolist()).to_dict()
        self.collection = []
        del self.collection[:]
        self._target_idx_list = {k : [] for k , v in self._target_index_dict_list.items()}
        self._target_list = comb[target_col].unique().tolist()

    def check_target_index_equal(self) :
        check_target = dict()
        for col in list(self._target_idx_list.keys()) :
            check_target[col] = len(self._target_idx_list[col]) == len(self._target_index_dict_list[col])
        else :
            print("check target index")
            self.print_pretty(check_target)
        if any([True for i in list(check_target.values()) if i == False]) :
            raise Exception("해당 컬럼으로 분리할 수 없음")


    # def find_use_col(self) :
    #     use_col = []
    #     for i in self.collection :
    #         for k , v in i.items() :
    #             use_col.extend(list(v.keys()))
    #     else :
    #         use_col = list(set(use_col))
    #     return use_col

    def get_condition(self,verbose= True) :
        self.check_target_index_equal()
        
        target_q = {i : [] for i in self._target_list}
        target_q_unique_list = {i : [] for i in self._target_list}
        target_q_unique_list_cond = {i : [] for i in self._target_list}
        for d in self.collection :
            for k , v in d.items() :
                target_q[k].append(v)
        for target , v in target_q.items() :
            result = []
            for i in v : 
                q = self.make_where_query(i)
                target_idx = self._comb.query(q).index.tolist()
                if any([True for i in target_idx if i in target_q_unique_list[target] ]) :        
                    continue 
                else :    
                    target_q_unique_list_cond[target].append(i)
                    target_q_unique_list[target].extend(target_idx)    
        use_col_list = self._find_use_col(target_q_unique_list_cond)
        result = dict(use_col_list = use_col_list , condition = target_q_unique_list_cond)
        if verbose :
            self.print_pretty(result )
        return result 
    
    def _find_use_col(self, target_condition) :
        use_col = []
        for k, conds in target_condition.items() :
            for cond in conds :
                use_col.extend(list(cond.keys()))
        else :
            use_col = list(set(use_col))
        return use_col


    def fit(self,comb , cols , target_col, previous_condition=[]) :
        if len(cols) == 0 :
            return print("Failed")
        result = make_gini_attributes(comb , cols =cols,target_col=target_col)
        
        select_col = get_min_giniattributes(result)[0]
        minthreshold = get_min_giniattributes(result)[1]
        
        find_pure_result = dict(filter(lambda x : x[1] == 0.0 , result.items()))
        if len(find_pure_result) == 0 :
            select_candidate = list(comb[select_col].unique())
            qs = [f"{select_col} == '{i}'" for i in select_candidate]        
            qs_dict = [{select_col : [i]} for i in select_candidate]        
            
            _cols = cols 
            for idx , q in enumerate(qs) :
                _comb = comb.query(q)
                _cols = list(set(_cols).difference(set([select_col])))
                previous_condition.append(qs_dict[idx])
                self.fit(_comb , _cols , target_col,previous_condition)
        else :
            for select_col, gini_impurity in find_pure_result.items() : 
                r = comb.groupby([target_col]).apply(lambda x : {select_col:list(x[select_col].unique())})
                
                r= r.to_dict()
                for k , v in r.items() :
                    for cond in previous_condition:
                        v.update(cond)
                
                self.collection.append(r)
                final_result = []
                for k , i in r.items() :
                    q = self.make_where_query(i)
                    _result = comb.query(q)
                    result:pd.DataFrame = _result.groupby([target_col]).apply(lambda x : x.index.tolist())
                    if result.empty()
                    if len(result) == 0 :
                        print("")
                        continue
                    for k, v in result.to_dict().items() :
                        self._target_idx_list[k] = sorted(list(set(v) | set(self._target_idx_list[k])))
            else :
                del previous_condition[-1]

cols = ["cola","colb","colc","cold"]
sc = SeperateCode()
sc.initialize(comb, target_col)
sc.fit(comb , cols ,"target")
# print(sc.find_condition())
print(sc.get_condition())
# print(sc.find_use_col())

target_aa = dict(cola=["A"],colc=["a","c"])
target_bb = dict(cola=["A"],colc=["b","d"])
target_cc = dict(cola=["C"],colc=["a","b"])
target_cc = dict(cola=["C"],colb=[1,4])
target_dd = dict(cola=["B"],colb=[1,2])
target_dd = dict(cola=["B"],colc=["e","b"])
target_ee = dict(cola=["B"],colc=["a","d"])

check target index
aa
	True
bb
	True
cc
	True
dd
	True
ee
	True
use_col_list
	['cola', 'colc', 'colb']
condition
	aa
		[{'colc': ['a', 'c'], 'cola': ['A']}]
	bb
		[{'colc': ['b', 'd'], 'cola': ['A']}]
	cc
		[{'colc': ['a', 'b'], 'cola': ['C']}]
	dd
		[{'colb': [1, 2], 'cola': ['B']}]
	ee
		[{'colb': [3, 4], 'cola': ['B']}]
{'use_col_list': ['cola', 'colc', 'colb'], 'condition': {'aa': [{'colc': ['a', 'c'], 'cola': ['A']}], 'bb': [{'colc': ['b', 'd'], 'cola': ['A']}], 'cc': [{'colc': ['a', 'b'], 'cola': ['C']}], 'dd': [{'colb': [1, 2], 'cola': ['B']}], 'ee': [{'colb': [3, 4], 'cola': ['B']}]}}


In [474]:

attribute_names =  ['age', 'income','student', 'credit_rate']
class_name = 'default'
data1 ={
    'age' : ['youth', 'youth', 'middle_age', 'senior', 'senior', 'senior','middle_age', 'youth', 'youth', 'senior', 'youth', 'middle_age','middle_age', 'senior'],
    'income' : ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium','low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student' : ['no','no','no','no','yes','yes','yes','no','yes','yes','yes','no','yes','no'],
    'credit_rate' : ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair','excellent', 'excellent', 'fair', 'excellent'],
    'default' : ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes','yes', 'yes', 'yes', 'no']
}
df1 = pd.DataFrame (data1, columns=data1.keys())
sc = SeperateCode()
sc.initialize(df1,class_name)
sc.fit(df1 , attribute_names ,class_name)
print(sc.collection)
#print(sc.get_condition())


[{'no': {'student': ['no'], 'age': ['youth']}, 'yes': {'student': ['yes'], 'age': ['youth']}}, {'yes': {'student': ['no', 'yes'], 'age': ['middle_age']}}, {'yes': {'credit_rate': ['fair', 'excellent'], 'age': ['middle_age']}}, {'yes': {'income': ['high', 'low', 'medium'], 'age': ['middle_age']}}, {'no': {'credit_rate': ['excellent'], 'age': ['senior']}, 'yes': {'credit_rate': ['fair'], 'age': ['senior']}}]


In [475]:
pd.DataFrame(sc.collection)

,no,yes
0,"{'student': ['no'], 'age': ['youth']}","{'student': ['yes'], 'age': ['youth']}"
1,NaN,"{'student': ['no', 'yes'], 'age': ['middle_age']}"
2,NaN,"{'credit_rate': ['fair', 'excellent'], 'age': ..."
3,NaN,"{'income': ['high', 'low', 'medium'], 'age': [..."
4,"{'credit_rate': ['excellent'], 'age': ['senior']}","{'credit_rate': ['fair'], 'age': ['senior']}"


In [476]:

print(sc.get_condition())

check target index
no
	True
yes
	True
use_col_list
	['student', 'age', 'credit_rate']
condition
	no
		[{'student': ['no'], 'age': ['youth']}, {'credit_rate': ['excellent'], 'age': ['senior']}]
	yes
		[{'student': ['yes'], 'age': ['youth']}, {'student': ['no', 'yes'], 'age': ['middle_age']}, {'credit_rate': ['fair'], 'age': ['senior']}]
{'use_col_list': ['student', 'age', 'credit_rate'], 'condition': {'no': [{'student': ['no'], 'age': ['youth']}, {'credit_rate': ['excellent'], 'age': ['senior']}], 'yes': [{'student': ['yes'], 'age': ['youth']}, {'student': ['no', 'yes'], 'age': ['middle_age']}, {'credit_rate': ['fair'], 'age': ['senior']}]}}


In [477]:

sc._target_idx_list

{'no': [0, 1, 5, 7, 13], 'yes': [2, 3, 4, 6, 8, 9, 10, 11, 12]}

In [478]:

attribute_names =  [ 'default','student', 'credit_rate']
class_name = 'age'
sc.initialize(df1,class_name)
print(sc._target_idx_list)
sc.fit(df1 , attribute_names ,class_name)
print(sc.get_condition())
#result = pd.DataFrame(sc.collection)
#sc.check_target_index_equal()


{'middle_age': [], 'senior': [], 'youth': []}
Failed
Failed
Failed
Failed
Failed
Failed
check target index
middle_age
	False
senior
	False
youth
	False


Exception: 해당 컬럼으로 분리할 수 없음

In [419]:
sc._target_idx_list

{'high': [1], 'low': [], 'medium': []}

In [338]:
target_q = {i : [] for i in sc._target_list}
target_q_unique_list = {i : [] for i in sc._target_list}
target_q_unique_list_cond = {i : [] for i in sc._target_list}
for d in sc.collection :
    for k , v in d.items() :
        target_q[k].append(v)
for target , v in target_q.items() :
    result = []
    for i in v : 
        q = sc.make_where_query(i)
        target_idx = df1.query(q).index.tolist()
        if any([True for i in target_idx if i in target_q_unique_list[target] ]) :        
            continue 
        else :    
            target_q_unique_list_cond[target].append(i)
            target_q_unique_list[target].extend(target_idx)    


{'no': [{'student': ['no'], 'age': ['youth']},
  {'credit_rate': ['excellent'], 'age': ['senior']}],
 'yes': [{'student': ['yes'], 'age': ['youth']},
  {'student': ['no', 'yes'], 'age': ['middle_age']},
  {'credit_rate': ['fair'], 'age': ['senior']}]}

In [340]:
for i in target_q_unique_list_cond["no"] : 
    q = sc.make_where_query(i)
    target_idx = df1.query(q).index.tolist()
    print(target_idx)
for i in target_q_unique_list_cond["yes"] : 
    q = sc.make_where_query(i)
    target_idx = df1.query(q).index.tolist()
    print(target_idx)

[0, 1, 7]
[5, 13]
[8, 10]
[2, 6, 11, 12]
[3, 4, 9]


In [316]:
for i in pd.DataFrame(sc.collection)["no"].dropna() :
    q = sc.make_where_query(i)
    print(df1.query(q).index.tolist())

[0, 1, 7]
[5, 13]


In [319]:
pd.DataFrame(sc.collection)

,no,yes
0,"{'student': ['no'], 'age': ['youth']}","{'student': ['yes'], 'age': ['youth']}"
1,NaN,"{'student': ['no', 'yes'], 'age': ['middle_age']}"
2,NaN,"{'credit_rate': ['fair', 'excellent'], 'age': ..."
3,NaN,"{'income': ['high', 'low', 'medium'], 'age': [..."
4,"{'credit_rate': ['excellent'], 'age': ['senior']}","{'credit_rate': ['fair'], 'age': ['senior']}"


In [317]:
sc.collection

[{'no': {'student': ['no'], 'age': ['youth']},
  'yes': {'student': ['yes'], 'age': ['youth']}},
 {'yes': {'student': ['no', 'yes'], 'age': ['middle_age']}},
 {'yes': {'credit_rate': ['fair', 'excellent'], 'age': ['middle_age']}},
 {'yes': {'income': ['high', 'low', 'medium'], 'age': ['middle_age']}},
 {'no': {'credit_rate': ['excellent'], 'age': ['senior']},
  'yes': {'credit_rate': ['fair'], 'age': ['senior']}}]

In [225]:

for  i in result["yes"] :
    total_q = []
    for k , v in i.items() :
        if len(v) == 1 :
            q = f"{k} == '{v[0]}'" 
        else :
            _comb = list(v)
            q = f"{k} in {_comb}"
        total_q.append(q)
    else :
        total_q = " and ".join([f"({i})" for i in total_q])
        #print(total_q)
        print(df1.query(total_q).index.tolist())

[8, 10]
[2, 6, 11, 12]
[2, 6, 11, 12]
[2, 6, 11, 12]
[3, 4, 9]


In [230]:
df1.groupby(["default"]).apply(lambda x : x.index.tolist()).to_dict()

{'no': [0, 1, 5, 7, 13], 'yes': [2, 3, 4, 6, 8, 9, 10, 11, 12]}

In [ ]:

df1.query("default == 'yes'").index.tolist()

In [201]:
df1.query("(credit_rate =='excellent') & (age == 'senior')")

,age,income,student,credit_rate,default
5,senior,low,yes,excellent,no
13,senior,medium,no,excellent,no


In [200]:
df1.query("default == 'no'")

,age,income,student,credit_rate,default
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
13,senior,medium,no,excellent,no


In [166]:
select_candidate = list(comb[select_col].unique())
select_candidate

['A', 'C', 'B']

In [145]:
qs = [f"{select_col} == '{i}'" for i in select_candidate]
print(qs[0])
comb2 = comb.query(qs[0])
comb2

cola == 'A'


,cola,colb,colc,target
0,A,1,a,aa
1,A,1,c,aa
2,A,2,a,aa
3,A,2,c,aa
4,A,3,a,aa
5,A,3,c,aa
6,A,2,b,bb
7,A,2,d,bb
8,A,3,b,bb
9,A,3,d,bb


In [148]:
cols = ["colb","colc"]
result = make_gini_attributes(comb2 , cols =cols,target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)

{'colb': 0.2857142857142857, 'colc': 0.0}
colc


In [149]:
select_candidate = list(comb2[select_col].unique())
select_candidate

['a', 'c', 'b', 'd']

In [150]:
print(qs[0])
comb.query(qs[0]).groupby(["target"]).apply(lambda x : list(x["colc"].unique()))


cola == 'A'


target
aa    [a, c]
bb    [b, d]
dtype: object

In [151]:
target_aa = dict(cola=["A"],colc=["a","c"])
target_bb =dict(cola=["A"],colc=["b","d"])

In [157]:
print(qs[1])
comb2 = comb.query(qs[1])
comb2
cols = ["colb","colc"]
result = make_gini_attributes(comb2 , cols =cols,target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)


cola == 'C'
{'colb': 0.0, 'colc': 0.0}
colb


In [158]:
print(qs[1])
comb.query(qs[1]).groupby(["target"]).apply(lambda x : list(x["colc"].unique()))

cola == 'C'


target
cc    [a, b]
dtype: object

In [159]:
print(qs[1])
comb.query(qs[1]).groupby(["target"]).apply(lambda x : list(x["colb"].unique()))


cola == 'C'


target
cc    [1, 4]
dtype: object

In [160]:

target_aa = dict(cola=["A"],colc=["a","c"])
target_bb = dict(cola=["A"],colc=["b","d"])
target_cc = dict(cola=["C"],colc=["a","b"])
target_cc = dict(cola=["C"],colb=[1,4])


In [161]:
print(qs[2])
comb2 = comb.query(qs[2])
cols = ["colb","colc"]
result = make_gini_attributes(comb2 , cols =cols,target_col=target_col)
print(result)
select_col = get_min_giniattributes(result)[0]
print(select_col)


cola == 'B'
{'colb': 0.5, 'colc': 0.0}
colc


In [162]:
print(qs[2])
comb.query(qs[2]).groupby(["target"]).apply(lambda x : list(x["colc"].unique()))

cola == 'B'


target
dd    [e, b]
ee    [a, d]
dtype: object

In [163]:
# print(qs[2])
# comb.query(qs[2]).groupby(["target"]).apply(lambda x : list(x["colc"].unique()))


In [164]:
target_aa = dict(cola=["A"],colc=["a","c"])
target_bb = dict(cola=["A"],colc=["b","d"])
target_cc = dict(cola=["C"],colc=["a","b"])
target_cc = dict(cola=["C"],colb=[1,4])
target_dd = dict(cola=["B"],colb=[1,2])
target_dd = dict(cola=["B"],colc=["e","b"])
target_ee = dict(cola=["B"],colc=["a","d"])

In [138]:
ASET = dict(
    cola = ["A"],
    colb = [1,2,3],
    colc = ["a","c"],
)
BSET = dict(
    cola = ["A"],
    colb = [2,3,4,5],
    colc = ["b","d"],
)
CSET = dict(
    cola = ["C"],
    colb = [1,4],
    colc = ["a","b"],
)
DSET = dict(
    cola = ["B"],
    colb = [1,2],
    colc = ["e","b"],
)